In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('data/NVDA_30min_sample.csv')

In [3]:
data

,timestamp,open,high,low,close,volume
0,2024-02-26 04:00:00,790.90,798.00,787.87,797.98,27109
1,2024-02-26 04:30:00,798.01,801.00,797.01,800.50,37848
2,2024-02-26 05:00:00,800.07,800.07,797.30,798.70,8654
3,2024-02-26 05:30:00,798.63,798.63,797.12,797.23,7711
4,2024-02-26 06:00:00,797.20,797.97,796.61,797.71,6169
...,...,...,...,...,...,...
347,2024-03-11 17:30:00,858.00,858.90,856.52,857.12,22844
348,2024-03-11 18:00:00,857.34,866.45,857.34,866.45,114907
349,2024-03-11 18:30:00,866.50,867.80,863.93,866.50,67558
350,2024-03-11 19:00:00,866.76,870.04,865.04,868.48,90359
